<a href="https://colab.research.google.com/github/somak135/Pattreg/blob/main/pattrec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
## prepare training data

import pandas as pd
import io
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['arcene_train.csv']))
df.drop('Unnamed: 0', inplace = True, axis = 1)
df.drop('V10001', inplace = True, axis = 1)

Saving arcene_train.csv to arcene_train.csv


In [13]:
## prepare validation data

uploaded = files.upload()
df2 = pd.read_csv(io.BytesIO(uploaded['arcene_valid.csv']))
df2.drop('Unnamed: 0', inplace = True, axis = 1)
df2.drop('V10001', inplace = True, axis = 1)

Saving arcene_valid.csv to arcene_valid.csv


In [40]:
## fit logistic model on training data
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(max_iter=100000)
X = df.drop('y', inplace = False, axis = 1) ##model matrix
Y = df['y'] ##target variable
logit.fit(X,Y)

LogisticRegression(max_iter=100000)

In [41]:
### accuracy of logistic on validation set
from sklearn.metrics import accuracy_score

X2 = df2.drop('y', inplace = False, axis = 1)
Y2 = df2['y']
Y2_pred_logit = logit.predict(X2)
accuracy_score(Y2, Y2_pred_logit)

0.84

In [42]:
## fit lda on training data
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X, Y)

LinearDiscriminantAnalysis()

In [43]:
### accuracy of lda on validation set

Y2_pred_lda = lda.predict(X2)
accuracy_score(Y2, Y2_pred_lda)

0.79

In [44]:
## fit qda on training data
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda.fit(X, Y)

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

In [45]:
### accuracy of qda on validation set

Y2_pred_qda = qda.predict(X2)
accuracy_score(Y2, Y2_pred_qda)

0.5

In [46]:
## fit svm on training data

from sklearn.svm import SVC
svm = SVC(kernel='linear', random_state = 1)
svm.fit(X, Y)

SVC(kernel='linear', random_state=1)

In [47]:
### accuracy of svm on validation set

Y2_pred_svm = svm.predict(X2)
accuracy_score(Y2, Y2_pred_svm)

0.83

In [48]:
## fit svm_rbf on training data

from sklearn.svm import SVC
svm_rbf = SVC(kernel='rbf', random_state = 1)
svm_rbf.fit(X, Y)

SVC(random_state=1)

In [49]:
### accuracy of svm_rbf on validation set

Y2_pred_svm_rbf = svm_rbf.predict(X2)
accuracy_score(Y2, Y2_pred_svm_rbf)

0.72

In [50]:
## fit svm_poly on training data

from sklearn.svm import SVC
svm_poly = SVC(kernel='poly', random_state = 1)
svm_poly.fit(X, Y)

SVC(kernel='poly', random_state=1)

In [51]:
### accuracy of svm on validation set

Y2_pred_svm_poly = svm_poly.predict(X2)
accuracy_score(Y2, Y2_pred_svm_poly)

0.79

In [52]:
### cross validation of svm with n-folds after stitching the two datasets
from sklearn.model_selection import cross_val_score

df_combined = pd.concat([df, df2])
X_combined = df_combined.drop('y', inplace = False, axis = 1)
Y_combined = df_combined['y']

n = 10
cv_scores_svm = cross_val_score(svm, X_combined, Y_combined, scoring="accuracy", cv = n)
print(cv_scores_svm, cv_scores_svm.mean(), cv_scores_svm.std())

[0.85 0.9  0.95 0.95 0.95 0.95 0.85 0.95 0.95 0.85] 0.9149999999999998 0.04499999999999999


In [53]:
### cross validation of logistic with n-folds after stitching the two datasets
n = 10
cv_scores_logit = cross_val_score(logit, X_combined, Y_combined, scoring="accuracy", cv = n)
print(cv_scores_logit, cv_scores_logit.mean(), cv_scores_logit.std())

[0.85 0.9  0.95 0.95 0.95 0.95 0.85 0.95 0.95 0.85] 0.9149999999999998 0.04499999999999999


In [54]:
### cross validation of svm with l1 penalty with n-folds after stitching the two datasets
from sklearn.svm import LinearSVC
svm_l1 = LinearSVC(penalty = 'l1', dual = False, max_iter=100000)

n = 10
cv_scores_svm_l1 = cross_val_score(svm_l1, X_combined, Y_combined, scoring="accuracy", cv = n)
print(cv_scores_svm_l1, cv_scores_svm_l1.mean(), cv_scores_svm_l1.std())

[0.8  0.7  0.75 0.9  0.8  0.75 0.9  0.8  0.85 0.85] 0.8099999999999999 0.062449979983984


In [61]:
#### PCA
from sklearn.preprocessing import StandardScaler
features = df_combined.columns
# Separating out the features
x = df_combined.loc[:, features[1:]].values
# Separating out the target
y = df_combined.loc[:,['y']].values
x = StandardScaler().fit_transform(x)

In [69]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents)
print(principalDf)

              0            1            2   ...          97          98          99
0   -6467.383189 -1655.653957   477.718517  ...  176.366050   59.490867 -230.338612
1   -6646.708355 -2492.392586  -535.208461  ...   88.410660   11.959593   29.008082
2   -7135.998881 -1790.895033   851.344561  ...    6.017318  273.408926    1.503169
3   -5674.917981 -2369.183982  -516.935258  ...   54.363892   31.616847  -69.720548
4     139.503841  6814.931196   -72.376137  ...   70.249985  -57.529411  209.395854
..           ...          ...          ...  ...         ...         ...         ...
195  7460.174306 -2985.412083   638.013331  ...  217.265136 -419.042834  -85.909144
196  5686.148597 -2290.593998 -3931.747262  ... -104.403895  -18.286978 -136.522228
197    22.678817  7005.822582  1671.193978  ... -349.894953  -76.391962  116.331938
198  -732.443056  7110.915579  -289.681997  ...  -50.854991  457.582342  -19.511429
199   440.846889  6878.795720  -135.834550  ...  127.835929    4.935111 -100

In [70]:
from sklearn.model_selection import cross_val_score

df_combined = pd.concat([df, df2])
X_combined = df_combined.drop('y', inplace = False, axis = 1)
Y_combined = df_combined['y']

n = 10
cv_scores_svm = cross_val_score(svm, principalDf, Y_combined, scoring="accuracy", cv = n)
print(cv_scores_svm, cv_scores_svm.mean(), cv_scores_svm.std())

[0.85 0.95 0.85 0.9  0.8  0.85 0.9  0.8  0.85 0.75] 0.85 0.0547722557505166
